In [1]:
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt 
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score,confusion_matrix

In [2]:
trainData = pd.read_csv("dataset/train_features.csv")
testData = pd.read_csv("dataset/test_features.csv")
data_cols = np.array(trainData.columns)
data_cols

array(['letter_slant', 'line_slant', 'letter_size', 'margin_slope',
       'word_spacing', 'personality'], dtype=object)

In [3]:
trainData.head()

,letter_slant,line_slant,letter_size,margin_slope,word_spacing,personality
0,vertical,upperside,931.7,straight,small,Agreeableness
1,backward,lowerside,486.2,right,medium,Agreeableness
2,forward,baseline,112.8,straight,small,Agreeableness
3,forward,upperside,30.6,straight,small,Agreeableness
4,backward,baseline,152.9,straight,small,Agreeableness


In [4]:
testData.head()

,letter_slant,line_slant,letter_size,margin_slope,word_spacing,personality
0,vertical,baseline,421.2,left,small,Agreeableness
1,vertical,baseline,94.8,right,medium,Agreeableness
2,forward,baseline,1397.6,left,large,Agreeableness
3,vertical,baseline,84.2,right,large,Agreeableness
4,vertical,upperside,763.3,left,small,Agreeableness


In [5]:
k = int(input())
print("The value of k is:", k)

The value of k is: 5


In [6]:
trainData.isnull().sum()

letter_slant    0
line_slant      0
letter_size     0
margin_slope    0
word_spacing    0
personality     0
dtype: int64

In [7]:
testData.isnull().sum()

letter_slant    0
line_slant      0
letter_size     0
margin_slope    0
word_spacing    0
personality     0
dtype: int64

In [8]:
trainData = trainData.dropna(axis=0)
testData = testData.dropna(axis=0)

In [9]:
map_dict = {
    val: key
    for key, val in enumerate(trainData['letter_slant'].unique())
}
trainData['letter_slant'] = trainData['letter_slant'].map(map_dict)
testData['letter_slant'] = testData['letter_slant'].map(map_dict)

map_dict = {
    val: key
    for key, val in enumerate(trainData['line_slant'].unique())
}
trainData['line_slant'] = trainData['line_slant'].map(map_dict)
testData['line_slant'] = testData['line_slant'].map(map_dict)

map_dict = {
    val: key
    for key, val in enumerate(trainData['margin_slope'].unique())
}
trainData['margin_slope'] = trainData['margin_slope'].map(map_dict)
testData['margin_slope'] = testData['margin_slope'].map(map_dict)

map_dict = {
    val: key
    for key, val in enumerate(trainData['word_spacing'].unique())
}
trainData['word_spacing'] = trainData['word_spacing'].map(map_dict)
testData['word_spacing'] = testData['word_spacing'].map(map_dict)

In [10]:
trainData.head()

,letter_slant,line_slant,letter_size,margin_slope,word_spacing,personality
0,0,0,931.7,0,0,Agreeableness
1,1,1,486.2,1,1,Agreeableness
2,2,2,112.8,0,0,Agreeableness
3,2,0,30.6,0,0,Agreeableness
4,1,2,152.9,0,0,Agreeableness


In [11]:
testData.head()

,letter_slant,line_slant,letter_size,margin_slope,word_spacing,personality
0,0,2,421.2,2,0,Agreeableness
1,0,2,94.8,1,1,Agreeableness
2,2,2,1397.6,2,2,Agreeableness
3,0,2,84.2,1,2,Agreeableness
4,0,0,763.3,2,0,Agreeableness


In [12]:
testData1 = testData.drop('personality', axis=1)
testData1.head()

,letter_slant,line_slant,letter_size,margin_slope,word_spacing
0,0,2,421.2,2,0
1,0,2,94.8,1,1
2,2,2,1397.6,2,2
3,0,2,84.2,1,2
4,0,0,763.3,2,0


In [13]:
def euclideanDistance(instance1,instance2,length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return np.sqrt(distance)

In [14]:
import operator
def getNeighbours(trainingSet,testSet,k):
    distance = []
    length = len(testSet)
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testSet,trainingSet.loc[x].values,length)
        # dist = euclideanDistance(testSet,trainingSet[x],length)
        distance.append([trainingSet.loc[x].values, dist])
    distance.sort(key=operator.itemgetter(1))
    neighbours = []
    for x in range(k):
        neighbours.append(distance[x][0])
    return neighbours

In [15]:
correct = 0
for key in range(len(testData)):
    Neighbours = getNeighbours(trainData, testData1.loc[key].values, k)
    targets = []
    for n in Neighbours:
        targets.append(n[-1])
    max_target = max(targets, key=targets.count)
    if testData['personality'][key] == max_target:
        correct += 1

accuracy = (correct / len(testData)) * 100
print(f'Accuracy: {accuracy}%')

Accuracy: 43.18181818181818%


In [16]:
Neighbours = getNeighbours(trainData,testData1.loc[4].values,k)
targets = []
for neighbour in Neighbours:
    targets.append(neighbour[-1])
Neighbours

[array([2, 0, 767.5, 1, 2, 'Neuroticism'], dtype=object),
 array([0, 0, 769.4, 0, 1, 'Agreeableness'], dtype=object),
 array([2, 2, 773.6, 0, 0, 'Conscientiousness'], dtype=object),
 array([0, 2, 783.1, 2, 1, 'Openness'], dtype=object),
 array([2, 0, 788.7, 2, 1, 'Agreeableness'], dtype=object)]

In [17]:
targets

['Neuroticism',
 'Agreeableness',
 'Conscientiousness',
 'Openness',
 'Agreeableness']

In [18]:
max_target = max(targets, key=targets.count)
max_target

'Agreeableness'